In [2]:

import pandas as pd 
import numpy as np
import math
from src.utility.descriptive_statistics import descriptive_statistics
#from src.graph.graph import plot_time_series

df = pd.read_pickle('src/data/panel_data.pkl')
df["index"] = pd.to_datetime(df["index"])

# Filtrage des dates spécifiques (outliers)
outlier_dates = [pd.Timestamp('2001-09-11')]
df = df[~df['index'].isin(outlier_dates)]

df = df[(df['index'] >= '1988-02-01') & (df['index'] <= '2016-12-31')]

for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

,index,Maturity 1,Maturity 2,Maturity 3,Maturity 4,Maturity 5
0,1988-02-01,197.75,203.75,206.75,208.50,210.50
1,1988-02-02,198.25,203.75,206.75,207.50,209.75
2,1988-02-03,198.75,204.00,207.25,208.25,211.25
3,1988-02-04,202.75,207.75,210.75,212.50,216.00
4,1988-02-05,201.25,207.00,210.50,212.00,214.25
...,...,...,...,...,...,...
7288,2016-12-23,345.75,352.50,360.00,367.25,376.25
7289,2016-12-27,355.00,361.25,368.50,375.75,384.25
7290,2016-12-28,348.25,354.75,362.00,369.25,378.25
7291,2016-12-29,349.75,355.75,363.00,370.25,379.25


In [3]:
from datetime import datetime, timedelta
from calendar import monthrange

#Calculez les jours restants jusqu'à la fin de l'année sous forme de fraction et stockez-les dans une nouvelle colonne.
def get_t(date):
    bgn_of_df = datetime(1988, 1, 1)  
    delta = date - bgn_of_df
    return delta.days / 365.0

#df.reset_index(inplace=True)
df.reset_index(drop=True, inplace=True)

df['t'] = df['index'].apply(get_t)


def get_T(date):
    month_codes = {'H': 3, 'K': 5, 'N': 7, 'U': 9, 'Z': 12}
    contract_T = []

    for year in [date.year, date.year + 1]:
        for code, month in sorted(month_codes.items(), key=lambda x: x[1]):
            if len(contract_T) >= 5:
                break
            last_day = monthrange(year, month)[1] - 17
            contract_date = datetime(year, month, last_day)

            if date > contract_date - timedelta(days=14):
                continue

            contract_T.append(contract_date)

    if len(contract_T) == 5:
        return contract_T
    else:
        return None
#df['T_date'] = df['index'].apply(get_T)
df[['T1', 'T2', 'T3', 'T4', 'T5']] = df['index'].apply(lambda x: pd.Series(get_T(x)))
df

,index,Maturity 1,Maturity 2,Maturity 3,Maturity 4,Maturity 5,t,T1,T2,T3,T4,T5
0,1988-02-01,197.75,203.75,206.75,208.50,210.50,0.084932,1988-03-14,1988-05-14,1988-07-14,1988-09-13,1988-12-14
1,1988-02-02,198.25,203.75,206.75,207.50,209.75,0.087671,1988-03-14,1988-05-14,1988-07-14,1988-09-13,1988-12-14
2,1988-02-03,198.75,204.00,207.25,208.25,211.25,0.090411,1988-03-14,1988-05-14,1988-07-14,1988-09-13,1988-12-14
3,1988-02-04,202.75,207.75,210.75,212.50,216.00,0.093151,1988-03-14,1988-05-14,1988-07-14,1988-09-13,1988-12-14
4,1988-02-05,201.25,207.00,210.50,212.00,214.25,0.095890,1988-03-14,1988-05-14,1988-07-14,1988-09-13,1988-12-14
...,...,...,...,...,...,...,...,...,...,...,...,...
7287,2016-12-23,345.75,352.50,360.00,367.25,376.25,28.997260,2017-03-14,2017-05-14,2017-07-14,2017-09-13,2017-12-14
7288,2016-12-27,355.00,361.25,368.50,375.75,384.25,29.008219,2017-03-14,2017-05-14,2017-07-14,2017-09-13,2017-12-14
7289,2016-12-28,348.25,354.75,362.00,369.25,378.25,29.010959,2017-03-14,2017-05-14,2017-07-14,2017-09-13,2017-12-14
7290,2016-12-29,349.75,355.75,363.00,370.25,379.25,29.013699,2017-03-14,2017-05-14,2017-07-14,2017-09-13,2017-12-14


In [4]:
df_log= pd.DataFrame({
    "index": df["index"]
})

for i in range(1, 6):
    df_log[f'log price Maturity {i}'] = np.log(df[f'Maturity {i}'])  #log price
    
for i in range(1, 6):
    df_log[f'time to maturity {i}'] = (df[f'T{i}'] - df['index']).dt.days / 365 #ttm

df_log["t"] = df["t"]

df_log

,index,log price Maturity 1,log price Maturity 2,log price Maturity 3,log price Maturity 4,log price Maturity 5,time to maturity 1,time to maturity 2,time to maturity 3,time to maturity 4,time to maturity 5,t
0,1988-02-01,5.287004,5.316894,5.331510,5.339939,5.349486,0.115068,0.282192,0.449315,0.616438,0.868493,0.084932
1,1988-02-02,5.289529,5.316894,5.331510,5.335131,5.345916,0.112329,0.279452,0.446575,0.613699,0.865753,0.087671
2,1988-02-03,5.292048,5.318120,5.333926,5.338739,5.353042,0.109589,0.276712,0.443836,0.610959,0.863014,0.090411
3,1988-02-04,5.311974,5.336335,5.350673,5.358942,5.375278,0.106849,0.273973,0.441096,0.608219,0.860274,0.093151
4,1988-02-05,5.304548,5.332719,5.349486,5.356586,5.367144,0.104110,0.271233,0.438356,0.605479,0.857534,0.095890
...,...,...,...,...,...,...,...,...,...,...,...,...
7287,2016-12-23,5.845716,5.865051,5.886104,5.906043,5.930254,0.221918,0.389041,0.556164,0.723288,0.975342,28.997260
7288,2016-12-27,5.872118,5.889570,5.909441,5.928924,5.951293,0.210959,0.378082,0.545205,0.712329,0.964384,29.008219
7289,2016-12-28,5.852921,5.871413,5.891644,5.911474,5.935555,0.208219,0.375342,0.542466,0.709589,0.961644,29.010959
7290,2016-12-29,5.857219,5.874228,5.894403,5.914178,5.938196,0.205479,0.372603,0.539726,0.706849,0.958904,29.013699


In [5]:
# get data
#df_log = df_log[(df_log['index'] >= '2015-01-01') & (df_log['index'] <= '2016-12-31')]
df_log = df_log[(df_log['index'] >= '1988-02-01') & (df_log['index'] <= '2015-01-01')]
df_log.shape

(6788, 12)

In [6]:
# get state variable
import numpy as np
from scipy.optimize import minimize
from filterpy.kalman import KalmanFilter
import pandas as pd
from src.utility.constant import DELTA


# Fonction pour la composante saisonnière s(t)
def s_func(t):
    return -0.0228 * np.cos(2 * np.pi * t) + 0.0029 * np.cos(2 * np.pi * 2 * t) + \
           0.0081 * np.sin(2 * np.pi * t) + 0.0054 * np.sin(2 * np.pi * 2 * t)

# Fonction objective pour l'optimisation
def get_x_t(params, observations, times, maturities):
    mu, sigma, lambda1 = params
    residuals = []
    a = mu - 0.5 * sigma ** 2
    kf = KalmanFilter(dim_x=1, dim_z=5)
    kf.F = np.array([[1]])  # Matrice de transition d'état A
    kf.H = np.ones((5, 1))  # Matrice de mesure C
    kf.x = np.array([[0]])  # État initial X0
    kf.P = np.eye(1) * 1000  # Grande covariance initiale
    kf.Q = np.array([[DELTA * sigma ** 2]])  # Variance du processus
    kf.R = np.eye(5) * 0.01  # Bruit de mesure

    x_t = []
    for i, z in enumerate(observations):
        t = times.iloc[i]
        s_t = s_func(t)
        c_t = np.array([s_t + (mu + lambda1 - 0.5 * sigma ** 2) * maturity for maturity in maturities.iloc[i]])
        c_t = c_t.reshape(-1, 1)  # Garantir la taille 5x1
        z = z.reshape(-1, 1)
        kf.predict(u=np.array([a]))
        kf.update(z - c_t)  # Mise à jour avec la différence
        x_t.append(kf.x.flatten().tolist()[0])  # stocker x_t

    return x_t

# Préparation des variables pour la fonction get_x_t
observations = df_log.iloc[:,1:6].values
times = df_log["t"]
maturities = df_log.iloc[:,6:11] 

# les paramètres optimaux
param_opti = [0.0416, 0.1585, -0.1011]

x_t_to_use = get_x_t(param_opti, observations, times, maturities)
len(x_t_to_use)

6788

In [16]:
# Price prediction

def rmse(df, params, state_vars):
    mu, sigma, lambda1 = params
    n = len(df)
    simulated_prices = np.zeros((n, 5))  
    actual_prices = df.iloc[:, 1:6].values 
    print(actual_prices)

    for t in range(n):
        s_t = s_func(df.iloc[t]['t'])
        if t == 0:
            x_t = state_vars[0]  # x0
        else:
            x_t = state_vars[t-1] + (mu - 0.5 * sigma**2) 
        for m in range(5):
            ttm = df.iloc[t, 6+m]
            simulated_prices[t, m] = mu * df.iloc[t]['t'] + (mu - lambda1 + 0.5 * sigma**2) * ttm + x_t + s_t  
    print(simulated_prices)
    
    #calcul RMSE
    rmse_values = np.sqrt(np.mean(((simulated_prices - actual_prices) / actual_prices) ** 2, axis=0))

    return rmse_values  

rmse_values = rmse(df_log, param_opti, x_t_to_use)
print(f"RMSE values for each maturity: {rmse_values}")

[[5.28700361 5.31689375 5.33151033 5.33993904 5.34948565]
 [5.28952886 5.31689375 5.33151033 5.33513134 5.34591635]
 [5.29204775 5.31811999 5.33392579 5.33873928 5.35304227]
 ...
 [6.02284208 6.04322648 6.0591232  6.06552788 6.07936095]
 [6.00758393 6.02827852 6.04441272 6.05150076 6.06552788]
 [5.98393628 6.00573721 6.02223621 6.02888075 6.04263283]]
[[5.37995676 5.40590451 5.43185226 5.45780001 5.49693432]
 [5.40900483 5.43495258 5.46090033 5.48684808 5.52598239]
 [5.40815189 5.43409964 5.46004739 5.48599514 5.52512945]
 ...
 [7.2702922  7.29623995 7.3221877  7.34813545 7.38726976]
 [7.27200992 7.29795767 7.32390542 7.34985316 7.38898748]
 [7.27036601 7.29631376 7.32226151 7.34820926 7.38734357]]
RMSE values for each maturity: [0.12945177 0.13054111 0.13236899 0.13540783 0.13920838]
